In [109]:
import autograd.numpy as np
from autograd import grad

In [196]:
class ORNeuron():
    def __init__(self, n):
        self.n = n
        self.weights = np.zeros(n)
        
    def getWeights(self):
        return self.weights    
    
    def present(self, inputs):
#         z = np.sum(np.multiply(inputs, self.weights))
#         return 1 - np.power(np.e, -z)

        mus = np.array([np.power(self.weights[i], inputs[i]) for i in range(0, len(inputs))])
        return 1 - np.product(mus)
    
    def updateWeights(self, g):
        self.weights -= g
        
        for i in range(0, len(self.weights)):
            if self.weights[i] < 0:
                self.weights[i] = 0
        
    def setGrad(self, g):
        self.grad = g
        
    def applyGrad(self):
        self.updateWeights(self.grad)
        self.grad = None
        
    def __repr__(self):
        return str(self.weights)

In [197]:
class ANDNeuron():
    def __init__(self, n):
        self.weights = np.zeros(n)
        
    def getWeights(self):
        return self.weights
        
    def present(self, inputs):
#         mus = np.array([np.power(self.weights[i], inputs[i]) for i in range(0, len(inputs))])

        mus = []
        for i in range(0, len(inputs)):
            if i == 1:
                x = 1
            else:
                x = 1 - inputs[i]
            
            mus.append(np.power(self.weights[i], x))
    

        return np.product(np.array(mus))
        
#         i = 1.0 - inputs
#         i[0] = 1.0
        
#         z = np.sum(np.multiply(i, self.weights))
#         return np.power(np.e, -z)
    
    def updateWeights(self, g):
        self.weights -= g
        
        for i in range(0, len(self.weights)):
            if self.weights[i] < 0:
                self.weights[i] = 0
        
    def setGrad(self, g):
        self.grad = g
        
    def applyGrad(self):
        self.updateWeights(self.grad)
        self.grad = None
        
    def __repr__(self):
        return str(self.weights)

In [198]:
class CNFNetwork():
    def __init__(self, n, c):
        self.n = n
        self.c = c
        
        d = []
        for i in range(0, c):
            d.append(ORNeuron(n*2 + 1))
        
        self.disjunctions = np.array(d)
        
        if not c == 0:
            self.conjunction = ANDNeuron(c + 1)
        else:
            self.conjunction = ANDNeuron(n*2 + 1)
        
    def getDisjunctions(self):
        return self.disjunctions
    
    def getConjunction(self):
        return self.conjunction
    
    def fowardprop(self, inputs):
        actualIn = self.__convertInputs__(inputs)
        
        if not self.c == 0:
            dout = [1]
            for d in self.disjunctions:
                dout.append(d.present(actualIn))
            
            actualIn = dout
            
        return self.conjunction.present(np.array(actualIn))
    
    def __convertInputs__(self, inputs):
        actual = [1]
        
        for i in inputs:
            actual.append(i)
            actual.append(1-i)
        
        return np.array(actual)
    
    def __repr__(self):
        s = "Disjunctions -> "
        for d in self.disjunctions:
            s += (str(d) + ", ")
            
        s += ("\nConjunction -> " + str(self.conjunction) + "\n")
            
        return s

In [199]:
def SSE(network, data, targets):
    predictions = predict(network, data)
    return np.sum(np.power(np.subtract(targets, predictions), 2.0))
    
def predict(network, data):
    t = np.array([network.fowardprop(d) for d in data])
    return t
    

In [202]:
def trainCNFNetwork(data, targets, inputNodes, numC):
    network = CNFNetwork(inputNodes, numC)
    print(network)
    print("Initial Loss: ", SSE(network, data, targets))
    
    pterb = 0.0001
    
    for i in range(1, 10000):
        if i%1000 == 0:
            print("Iteration -> " + str(i) + " : " + str(SSE(network, data, targets)))
            
        for d in network.getDisjunctions():
            g = computeNeuronGrad(network, d, pterb, data, targets)
            d.setGrad(g * 0.1)
            
        g = computeNeuronGrad(network, network.getConjunction(), pterb, data, targets)
        
        network.getConjunction().updateWeights(g * 0.1)
        for d in network.getDisjunctions():
            d.applyGrad()
            
    print("Trained Loss: ", SSE(network, data, targets))
    return network

def computeNeuronGrad(network, neuron, pterb, data, targets):
    gradient = np.zeros(len(neuron.getWeights()))
    for k in range(0, len(neuron.getWeights())):
        g = np.zeros(len(neuron.getWeights()))
        g[k] = -pterb

        oldSSE = SSE(network, data, targets)
        neuron.updateWeights(g)
        newSSE = SSE(network, data, targets)
        neuron.updateWeights(-g)
                
        gradient[k] = (newSSE - oldSSE)/pterb
        
    return gradient

# AND
For this we dont neeed any disjunctions

In [ ]:
dataAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsAND = np.array([0.0, 0.0, 0.0, 1.0])
trainCNFNetwork(dataAND, targetsAND, 2, 0)

Disjunctions -> 
Conjunction -> [ 0.  0.  0.  0.  0.]

Initial Loss:  3.0
